In [1]:
import pandas as pd
import glob
import os

data_to_read_address = "data/ETF/BLOOMBERG_daily"
data_to_save_address = "data/ETF/BLOOMBERG_weekly"

csv_files = glob.glob(os.path.join(data_to_read_address, "*.csv"))

for file in csv_files:

    etf_name = file.split(data_to_read_address)[1].split('\\')[1].split('.csv')[0]
    etf_daily_df = pd.read_csv(file, header=None, names=['date',etf_name])
    
    etf_daily_df['date'] = pd.to_datetime(etf_daily_df['date'], format='%d/%m/%Y')

    # Set the 'date' column as the index
    etf_daily_df.set_index('date', inplace=True)

    # Resample to monthly frequency and calculate the monthly returns
    # Assuming 'returns' is the column name for daily returns in percentage form
    etf_daily_df[etf_name] = etf_daily_df[etf_name] / 100  # Convert to decimal form if needed

    # Calculate monthly returns by compounding daily returns within each month
    monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1
    col_name = etf_name+'_monthly_return'
    monthly_returns_df = monthly_returns.to_frame(name = col_name)
    monthly_returns_df.reset_index(inplace=True)

    # Convert monthly returns back to percentage if desired
    monthly_returns_df[col_name] *= 100
    monthly_returns_df[col_name] = monthly_returns_df[col_name].round(4)

    
    monthly_returns_df.to_csv(os.path.join(data_to_save_address,etf_name+ ".csv"),index=False)


C:\Users\ai123\AppData\Local\Temp\ipykernel_5160\1445425812.py:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1
C:\Users\ai123\AppData\Local\Temp\ipykernel_5160\1445425812.py:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1
C:\Users\ai123\AppData\Local\Temp\ipykernel_5160\1445425812.py:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1
C:\Users\ai123\AppData\Local\Temp\ipykernel_5160\1445425812.py:25: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1
C:\Users\ai123\AppData\Local\Temp\ipyker

In [58]:

etf_daily_df['date'] = pd.to_datetime(etf_daily_df['date'], format='%d/%m/%Y')

# Set the 'date' column as the index
etf_daily_df.set_index('date', inplace=True)

# Resample to monthly frequency and calculate the monthly returns
# Assuming 'returns' is the column name for daily returns in percentage form
etf_daily_df[etf_name] = etf_daily_df[etf_name] / 100  # Convert to decimal form if needed

# Calculate monthly returns by compounding daily returns within each month
monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1
col_name = etf_name+'_monthly_return'
monthly_returns_df = monthly_returns.to_frame(name = col_name)
monthly_returns_df.reset_index(inplace=True)

# Convert monthly returns back to percentage if desired
monthly_returns_df[col_name] *= 100
monthly_returns_df[col_name] = monthly_returns_df[col_name].round(4)
# Display the monthly returns
monthly_returns_df


C:\Users\ai123\AppData\Local\Temp\ipykernel_22748\1306251026.py:11: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_returns = (1 + etf_daily_df[etf_name]).resample('M').prod() - 1


,date,ISF LN Equity_monthly_return
0,2000-05-31,0.6827
1,2000-06-30,-1.6472
2,2000-07-31,0.5910
3,2000-08-31,6.7050
4,2000-09-30,-6.0019
...,...,...
290,2024-07-31,2.5318
291,2024-08-31,0.8108
292,2024-09-30,-1.5764
293,2024-10-31,-1.3764


In [59]:
monthly_returns_df.to_csv(os.path.join(data_to_save_address,etf_name+ ".csv"),index=False)